In [9]:
from sympy import *
from barfoot_utils_np import create_rot
init_printing()

In [10]:
# State
x, y, z, rho, phi, theta = symbols('x y z \\rho \\phi \\theta', real=True)
xn, yn, zn, rhon, phin, thetan = symbols('xn yn zn rhon phin thetan', real=True)

# Landmark
mx, my, mz = symbols('mx my mz', real=True)

# Control input
vx, vy, vz, wx, wy, wz = symbols('v_x v_y v_z \\omega_x \\omega_y \\omega_z', real=True)
dt = Symbol('dt', real=True)

# Vehicle state and landmark
X = Matrix([x,y,z, rho, phi, theta])
Xn = Matrix([xn,yn,zn, rhon, phin, thetan])
V = Matrix([vx,vy,vz, wx, wy, wz])
m = Matrix([mx, my, mz])

In [21]:
def create_rot_sym(Xrot):

    '''
    :param Xrot: Euler angles (1x3)
    '''

    # each dimension
    Rx = rot_axis1(Xrot[0])
    Ry = rot_axis2(Xrot[1])
    Rz = rot_axis3(Xrot[2])
    
    # Sympy doesn't follow the right-hand rule
    Rz[0,1] = -Rz[0,1]
    Rz[1,0] = -Rz[1,0]
    Ry[0,2] = -Ry[0,2]
    Ry[2,0] = -Ry[2,0]
    Rx[1,2] = -Rx[1,2]
    Rx[2,1] = -Rx[2,1]

    Rxyz = Rz.multiply(Ry).multiply(Rx)

    return Rxyz

In [22]:
def dynamics(X, V):

    '''
    :param X: latent state (1x6)
    :param V: control input (1x6)
    '''

    dV = Matrix([V[3:6]])
    Rxyz = create_rot_sym(X[3:6])
    dX = Rxyz.subs([
        (rho, X[3]),
        (phi, X[4]),
        (theta, X[5])
    ]).multiply(Matrix(V[0:3]))
    return Matrix([*dX, *dV])

dynamics(X, V)

⎡vₓ⋅cos(\phi)⋅cos(\theta) + v_y⋅(sin(\phi)⋅sin(\rho)⋅cos(\theta) - sin(\theta)
⎢                                                                             
⎢vₓ⋅sin(\theta)⋅cos(\phi) + v_y⋅(sin(\phi)⋅sin(\rho)⋅sin(\theta) + cos(\rho)⋅c
⎢                                                                             
⎢                                           -vₓ⋅sin(\phi) + v_y⋅sin(\rho)⋅cos(
⎢                                                                             
⎢                                                                        \omeg
⎢                                                                             
⎢                                                                        \omeg
⎢                                                                             
⎣                                                                        \omeg

⋅cos(\rho)) + v_z⋅(sin(\phi)⋅cos(\rho)⋅cos(\theta) + sin(\rho)⋅sin(\theta))⎤
                                                     

In [23]:
def measurment_model(X, L):

    '''
    :param X: latent state (1x6)
    :param L: landmarks
    '''

    T = eye(4)
    T[0:3, 0:3] = create_rot_sym(X[3:6])
    T[0:3, 3] = X[0:3]
    Tinv = eye(4)
    Tinv[0:3, 0:3] = T[0:3, 0:3].T
    Tinv[0:3, 3] = -T_inv[0:3,0:3]*T[0:3,3]

    return Tinv

measurment_model(X, None)

⎡                 cos(\phi)⋅cos(\theta)                                    sin
⎢                                                                             
⎢sin(\phi)⋅sin(\rho)⋅cos(\theta) - sin(\theta)⋅cos(\rho)  sin(\phi)⋅sin(\rho)⋅
⎢                                                                             
⎢sin(\phi)⋅cos(\rho)⋅cos(\theta) + sin(\rho)⋅sin(\theta)  sin(\phi)⋅sin(\theta
⎢                                                                             
⎣                           0                                                 

(\theta)⋅cos(\phi)                       -sin(\phi)       -x⎤
                                                            ⎥
sin(\theta) + cos(\rho)⋅cos(\theta)  sin(\rho)⋅cos(\phi)  -y⎥
                                                            ⎥
)⋅cos(\rho) - sin(\rho)⋅cos(\theta)  cos(\phi)⋅cos(\rho)  -z⎥
                                                            ⎥
       0                                      0           1 ⎦